In [ ]:
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import src.constants as constants

# Template and Styling Options

In [ ]:
grey_light = "rgb(224, 224, 224)"

axis_common_dict = dict(
    linecolor="black",
    linewidth=2,
    ticks="outside",
    title=dict(standoff=15),
)

colorway = px.colors.qualitative.G10
fillcolors = [px.colors.hex_to_rgb(x) + (0.15,)  for x in colorway]
fillcolors = [f"rgba{x}" for x in fillcolors]

template = go.layout.Template()
template.layout = dict(
    paper_bgcolor="white",
    plot_bgcolor=grey_light,
    font=dict(color="black"),
    title=dict(
        font=dict(size=18),
        x=0.1, xanchor="left",
        y=0.9, yanchor="top"
    ),
    xaxis=dict(
        **axis_common_dict,
    ),
    yaxis=dict(
        **axis_common_dict,
    )
)

# Model Selection

In [ ]:
title = "DQN - Encoding with Numerical Features"
save_title = "dqn_features"
model_names = [
    "DQN-n-m-xxlh-pmp30-g65-ftu5k",
    "DQN-nf-trainnorm-m",
    "DQN-nf-trainnorm-noeng-m",
]
model_legend_names = [
    "Without Features",
    "All Features",
    "Without Engagement",
]

# DDPG and TD3

In [ ]:
model_dirs = [
    os.path.join(constants.MODELS_PATH, m)
    for m in model_names
]
indices = [10_000, 100_000, 200_000, 1_000_000, 2_000_000, 
           3_000_000]
ticktext = ["10K", "", "200K", "1M", "2M", "3M"]

model_data = []
i = 0
suffix = ["a", "ac"]
for model_dir in model_dirs:
    seeds = [7, 42]
    data_paths = []
    for s in seeds:
        data_path = os.path.join(
            model_dir,
            f"predictions_{s}",
            f"eval_results_{suffix[i]}.txt"
        )
        if os.path.exists(data_path):
            data_paths.append(data_path)

    data_runs = []
    for path in data_paths:
        data_run = pd.read_csv(path, sep="\t")
        data_runs.append(data_run)

    model_data.append(data_runs)
    i += 1

In [ ]:
n_models = len(model_data)
model_results = []
for i in range(n_models):
    n_runs = len(model_data[i])
    returns = np.zeros((n_runs, 6))
    for j in range(n_runs):
        returns[j] = model_data[i][j]["mean_return"]
    model_results.append(returns)

# Other Models

In [ ]:
model_dirs = [
    os.path.join(constants.MODELS_PATH, m)
    for m in model_names
]
indices = [10_000, 100_000, 200_000, 1_000_000, 2_000_000, 
           3_000_000, 4_000_000, 5_000_000, 6_000_000]
ticktext = ["10K", "", "200K", "1M", "2M", "3M", "4M", "5M", "6M"]

model_data = []
for model_dir in model_dirs:
    seeds = [7, 42]
    data_paths = []
    for s in seeds:
        data_path = os.path.join(
            model_dir,
            f"predictions_{s}",
            "eval_results.txt"
        )
        if os.path.exists(data_path):
            data_paths.append(data_path)

    data_runs = []
    for path in data_paths:
        data_run = pd.read_csv(path, sep="\t")
        data_runs.append(data_run)

    model_data.append(data_runs)

In [ ]:
n_models = len(model_data)
model_results = []
for i in range(n_models):
    n_runs = len(model_data[i])
    returns = np.zeros((n_runs, 9))
    for j in range(n_runs):
        returns[j] = model_data[i][j]["mean_return"]
    model_results.append(returns)

# All Models

In [ ]:
model_results

In [ ]:
models_returns_mean = [
    np.mean(results, axis=0)
    for results in model_results
]
models_returns_std = [
    np.std(results, axis=0)
    for results in model_results
]

In [ ]:
models_returns_mean

In [ ]:
models_returns_std

# DDPG and TD3

In [ ]:
fig = go.Figure()

for i in range(n_models):
    returns_mean = models_returns_mean[i]
    returns_std = models_returns_std[i]

    fig.add_trace(
        go.Scatter(
            x=indices,
            y=returns_mean,
            mode="markers+lines",
            name=model_legend_names[i],
            marker=dict(
                size=4,
                color=colorway[i]
            ),
        )
    )

    fig.add_trace(
        go.Scatter(
            x=indices + indices[::-1],
            y=list(returns_mean + returns_std) + \
              list(returns_mean - returns_std)[::-1],
            fill="tozerox",
            fillcolor=fillcolors[i],
            line_color="rgba(255,255,255,0)",
            showlegend=False
        )
    )

argmax = np.argmax(models_returns_mean) % len(models_returns_mean[0])
best_model = np.argmax(models_returns_mean) // len(models_returns_mean[0])
max = np.max(models_returns_mean)
fig.add_annotation(
    x=indices[argmax],
    y=max,
    text=f"{model_legend_names[best_model]}: {max:.4f}",
    showarrow=True,
    arrowhead=6,
    arrowsize=0.75
)

fig.update_layout(
    title=title,
    xaxis=dict(
        tickangle=60,
        tickvals=indices,
        ticktext=ticktext,
        range=[0, 3_250_000],
        title="Training Steps"
    ),
    yaxis=dict(
        range=[0.55, 0.75],
        dtick=0.02,
        title="Discounted Return"
    ),
    width=850, height=500,
    template=template
)
fig.update_yaxes(template.layout.yaxis)
fig.update_xaxes(template.layout.xaxis)
fig.show()

# Other Models

In [ ]:
fig = go.Figure()

for i in range(n_models):
    returns_mean = models_returns_mean[i]
    returns_std = models_returns_std[i]

    fig.add_trace(
        go.Scatter(
            x=indices,
            y=returns_mean,
            mode="markers+lines",
            name=model_legend_names[i],
            marker=dict(
                size=4,
                color=colorway[i]
            ),
        )
    )

    fig.add_trace(
        go.Scatter(
            x=indices + indices[::-1],
            y=list(returns_mean + returns_std) + \
              list(returns_mean - returns_std)[::-1],
            fill="tozerox",
            fillcolor=fillcolors[i],
            line_color="rgba(255,255,255,0)",
            showlegend=False
        )
    )

argmax = np.argmax(models_returns_mean) % len(models_returns_mean[0])
best_model = np.argmax(models_returns_mean) // len(models_returns_mean[0])
max = np.max(models_returns_mean)
fig.add_annotation(
    x=indices[argmax],
    y=max,
    text=f"{model_legend_names[best_model]}: {max:.4f}",
    showarrow=True,
    arrowhead=6,
    arrowsize=0.75
)

fig.update_layout(
    title=title,
    xaxis=dict(
        tickangle=60,
        tickvals=indices,
        ticktext=ticktext,
        range=[0, 6_250_000],
        title="Training Steps"
    ),
    yaxis=dict(
        range=[0.62, 0.79],
        dtick=0.02,
        title="Discounted Return"
    ),
    width=850, height=500,
    template=template
)
fig.update_yaxes(template.layout.yaxis)
fig.update_xaxes(template.layout.xaxis)
fig.show()

In [ ]:
fig.write_image(f"./visualizations/results/{save_title}.pdf")